In [1]:
import numpy as np

In [2]:
def prep_lines(lines):
    for i in range(len(lines)):
        lines[i]=lines[i].replace('     * ', '')
        lines[i]=lines[i].replace(' \n', '')
        lines[i]=lines[i].replace('\n', '')
    return lines


def fortran_to_python(ODEs, spec):
    print(spec)

    ## split string
    ODEs[spec] = ODEs[spec].split(' ')

    ## remove empty strings
    to_remove = ''
    while to_remove in ODEs[spec]:
        ODEs[spec].remove('')

    ## replace brackets: ( --> [
    for i in range(len(ODEs[spec])):
        if ODEs[spec][0] == 'F' or ODEs[spec][0] == 'D':
            ODEs[spec][i] = ODEs[spec][i].replace('(', '[')
            ODEs[spec][i] = ODEs[spec][i].replace(')', ']')

    ## replace final line
    last = ODEs[spec][-1]
    # print('last', last)
    while not last[0] == 'Y':
        ODEs[spec].remove(last)
        last = ODEs[spec][-1]
    # print('at the end',last)
    for i in range(5,len(last)):
        if last[i] == ')':
            stop = i
            break
    nb = last[5:stop]

    ODEs[spec][-1] = 'YDOT['+nb+']=F-(D*Y['+nb+'])'

    return ODEs

    

In [3]:
file = '/lhome/silkem/CHEM/src-IP-AP-HNR/code/acodes.f'


with open(file, 'r') as f:
    lines = f.readlines()

type(lines)

list

In [4]:
np.shape(lines)

(10519,)

In [5]:
spec_lines = lines[123:]

In [6]:
spec_lines = prep_lines(spec_lines)

In [7]:
spec_lines[0]

'C H'

In [8]:
ODEs = dict()

for i, line in enumerate(spec_lines):
    if len(line) != 0 and line[0] == 'C':
        ODEs[line[2:]] = str('')
        print('i=',i, end = '\r')
        start = i + 1

        for j in range(start,start+1000):
            if j >= len(spec_lines):
                break
            if len(spec_lines[j]) != 0 and spec_lines[j][0] == 'C':
                # print(j, spec_lines[j])
                stop = j
                break
            
        for j in range(i+1,stop):
            ODEs[line[2:]] += spec_lines[j]

        i = stop +1



In [9]:
for key in ODEs:
    if key == 'CO':
        print(key)

CO


In [12]:
ODEs['H']

['F=0.+K(111)*Y(18)*Y(52)*HNR+K(129)*Y(28)*Y(52)*HNR+K(134)*X(2)*Y(14)*HNR+K(135)*X(2)*X(2)*HNR+K(135)*X(2)*X(2)*HNR+K(136)*X(2)*Y(37)*HNR+K(139)*X(2)*Y(31)*HNR+K(140)*X(2)*X(1)*HNR+K(140)*X(2)*X(1)*HNR+K(141)*Y(1)*Y(14)*HNR+K(141)*Y(1)*Y(14)*HNR+K(142)*Y(1)*X(2)*HNR+K(142)*Y(1)*X(2)*HNR+K(142)*Y(1)*X(2)*HNR+K(143)*Y(1)*Y(37)*HNR',
 'F=F+K(143)*Y(1)*Y(37)*HNR+K(144)*Y(1)*Y(104)*HNR+K(145)*Y(1)*Y(31)*HNR+K(145)*Y(1)*Y(31)*HNR+K(319)*Y(130)*Y(2)*HNR+K(320)*Y(2)*Y(451)*HNR+K(321)*Y(2)*Y(458)*HNR+K(322)*Y(2)*Y(455)*HNR+K(323)*Y(2)*Y(51)*HNR+K(324)*Y(2)*Y(56)*HNR+K(325)*Y(2)*Y(61)*HNR+K(326)*Y(2)*Y(74)*HNR+K(327)*Y(2)*Y(82)*HNR+K(328)*Y(2)*Y(238)*HNR+K(329)*Y(2)*Y(54)*HNR+K(330)*Y(2)*Y(141)*HNR+K(331)*Y(2)*Y(154)*HNR+K(332)*Y(2)*Y(314)*HNR+K(333)*Y(2)*Y(135)*HNR+K(334)*Y(2)*Y(146)*HNR+K(335)*Y(2)*Y(140)*HNR+K(336)*Y(2)*Y(280)*HNR+K(337)*Y(2)*Y(373)*HNR',
 'F=F+K(338)*Y(2)*Y(257)*HNR+K(339)*Y(2)*Y(274)*HNR+K(340)*Y(2)*Y(265)*HNR+K(341)*Y(2)*Y(399)*HNR+K(342)*Y(2)*Y(402)*HNR+K(343)*Y(2)*Y(326

In [11]:
for key in ODEs:
    ODEs = fortran_to_python(ODEs, key)

H



IndexError: list index out of range